# Introduction to DSGE.jl

Pearl Li <br> CEF 2017

June 27, 2017

## Outline

1. Notation
2. Solving, estimating, and forecasting an existing model
3. Model implementation
4. Exercise: MA(1) model
5. State-space routines
6. Conclusion

## Notation

- $y_t$ is a vector of observables at time $t$
- $s_t$ is a vector of states, including expectations of future states and lags
- $\epsilon_t$ is a vector of exogenous shocks
- $\eta_t$ is a vector of rational expectations errors
- $u_t$ is a vector of measurement error
- $\theta$ is a vector of parameters

Equilibrium conditions

$$\Gamma_0(\theta) s_t = \Gamma_1(\theta) s_{t-1} + \Psi(\theta) \epsilon_t + \Pi(\theta) \eta_t + C(\theta)$$

which are solved to give the state-space representation

$$
\begin{align*}
s_t &= T(\theta) s_{t-1} + R(\theta) \epsilon_t + C(\theta) & \epsilon_t \sim N(0, Q(\theta)) \\
y_t &= Z(\theta) s_t + D(\theta) + u_t & u_t \sim N(0, E(\theta))
\end{align*}
$$

## Using Existing Models

Let's construct an instance of the following (log-linearized) three-equation New Keynesian model:

$$
\begin{align*}
y_t - g_t &= -\frac{1}{\tau} R_t + \frac{1}{1 + \tau} \mathbb{E} \pi_{t+1} + \mathbb{E}(y_{t+1} - g_{t+1}) + \frac{1}{\tau} \mathbb{E} z_{t+1} \\
\pi_t &= \beta \mathbb{E} \pi_{t+1} + \kappa (y_t - g_t) \\
R_t &= \rho_R R_{t-1} + (1 - \rho_R) [\psi_1 \pi_t + \psi_2 (y_t - g_t)] + \epsilon_{R,t}
\end{align*}
$$

where

$$
\begin{align*}
s_t &= [y_t, \pi_t, R_t, y_{t-1}, g_t, z_t, \mathbb{E} y_{t+1}, \mathbb{E} \pi_{t+1}]' \\
y_t &= [\text{Real per-capita GDP growth}_t, \text{CPI inflation}_t, \text{Nominal FFR}_t]'
\end{align*}
$$

are the state and observable vectors respectively.

In [ ]:
using DSGE

# Construct model object
m = AnSchorfheide()

# Set data vintage and initial forecast date
m <= Setting(:data_vintage, "170528")
m <= Setting(:date_forecast_start, DSGE.quartertodate("2017-Q2"))

# Set input and output directories
m <= Setting(:dataroot, joinpath(pwd(), "input_data"))
m <= Setting(:saveroot, pwd())

# Don't use population forecast
m <= Setting(:use_population_forecast, false)

In [ ]:
m

In [ ]:
# Read in a previously computed vector of modal parameters
mode_file = rawpath(m, "estimate", "paramsmode.h5")
specify_mode!(m, mode_file)

In [ ]:
# Solve the rational expectations model to get transition equation 
# matrices
TTT, RRR, CCC = solve(m)

Estimate and forecast the model:

In [ ]:
# Find a posterior mode, sample from the posterior distribution
estimate(m)

# Add parallel workers
my_procs = addprocs(50)

# Forecast using the full distribution of parameters
output_vars = [:forecaststates, :forecastobs]
forecast_one(m, :full, :none, output_vars)
means_bands_all(m, :full, :none, output_vars)

# Remove parallel workers
rmprocs(my_procs)

Since in practice estimating and forecasting the full distribution is time-consuming, we'll forecast only at the mode using the mode we read in:

In [ ]:
# Load data
df = load_data(m)

# Forecast using modal parameters
output_vars = [:histobs, :forecaststates, :forecastobs]
forecast_one(m, :mode, :none, output_vars, df = df, verbose = :none)
means_bands_all(m, :mode, :none, output_vars, verbose = :none)

In [ ]:
# Read in forecasted observables
files = get_meansbands_output_files(m, :mode, :none, output_vars)
mb = read_mb(files[:forecastobs])

In [ ]:
# Show first four forecasted quarters
mb.means[1:4, :]

In [ ]:
# Read in forecasted states
mb = read_mb(files[:forecaststates])

# Show first forecasted quarters of output, inflation, and interest rate
mb.means[1:4, [:y_t, :π_t, :R_t]]

In [ ]:
using Plots
include("util.jl")
include("plot_history_and_forecast.jl")

# Plot nominal rate history and modal forecast
hist_mb  = read_mb(files[:histobs])
fcast_mb = read_mb(files[:forecastobs])
plot_history_and_forecast(:obs_nominalrate, hist_mb, fcast_mb)

## The Model Object

Subtypes of `AbstractModel` contain the following fields:

In [ ]:
fieldnames(m)

These fields include:

- Vectors of time-invariant (`parameters`) and steady-state parameters (`steady_state`)
- `Dict{Symbol, Int}`s mapping states, shocks, expectational errors, equations, or observables to indices. For example:

In [ ]:
m.endogenous_states

`m.endogenous_states[:y_t] = 1` indicates that the first element of the state vector

$$s_t = [y_t, \pi_t, R_t, y_{t-1}, g_t, z_t, \mathbb{E} y_{t+1}, \mathbb{E} \pi_{t+1}]$$

is $y_t$.

In [ ]:
m.equilibrium_conditions

`m.equilibrium_conditions[:eq_euler] = 1` means the first row of the equilibrium conditions

$$\Gamma_0 s_t = \Gamma_1 s_{t-1} + \Psi \epsilon_t + \Pi \eta_t + C$$

is the consumption Euler equation.

Additional fields in the model object:

- Strings giving the model specification (`spec`) and subspecification (`subspec`)
- `Dict{Symbol, Setting}`s of model settings, both for regular use (`settings`) and testing the package (`test_settings`)

## Exercise: MA(1) Model

See `exercise.ipynb`

## State-Space Routines

[StateSpaceRoutines.jl](https://github.com/FRBNY-DSGE/StateSpaceRoutines.jl)

- Package implementing **DSGE.jl-agnostic** state-space routines
- DSGE.jl functions `filter` and `smooth` wrap calls to these functions

Implemented routines:

- Kalman filter (`kalman_filter`)
- Kalman smoothers:
  + `hamilton_smoother`: James Hamilton, [_Time Series Analysis_](https://www.amazon.com/Time-Analysis-James-Douglas-Hamilton/dp/0691042896) (1994)
  + `koopman_smoother`: S.J. Koopman, ["Disturbance Smoother for State Space Models"](https://www.jstor.org/stable/2336762) (_Biometrika_, 1993)

- Simulation smoothers:
  + `carter_kohn_smoother`: C.K. Carter and R. Kohn, ["On Gibbs Sampling for State Space Models"](https://www.jstor.org/stable/2337125) (_Biometrika_, 1994)
  + `durbin_koopman_smoother`: J. Durbin and S.J. Koopman, ["A Simple and Efficient Simulation Smoother for State Space Time Series Analysis"](https://www.jstor.org/stable/4140605) (_Biometrika_, 2002)

In [ ]:
using StateSpaceRoutines

In [ ]:
?kalman_filter

Example: Kalman filter

In [ ]:
# Compute state-space matrices
sys = compute_system(m)

# Convert DataFrame to matrix
data = df_to_matrix(m, df)

# Call Kalman filter
loglike, s_T, P_T, _, _, _, _, _, _, _, _, s_0, P_0 =
    kalman_filter(data, sys[:TTT], sys[:RRR], sys[:CCC], sys[:QQ],
                  sys[:ZZ], sys[:DD], sys[:EE])
loglike

In [ ]:
s_T

Example: simulation smoother

In [ ]:
# Call simulation smoother
smoothed_states, smoothed_shocks =
    durbin_koopman_smoother(data, sys[:TTT], sys[:RRR], sys[:CCC], sys[:QQ],
                            sys[:ZZ], sys[:DD], sys[:EE], s_0, P_0,
                            draw_states = true)
smoothed_states

In [ ]:
smoothed_shocks

## Conclusion

Things learned:

- We love open source (and Julia)!
  + Open-source languages and packages reduce costs of writing code and make it easier to share
  + Julia is high-performance and high-productivity
- Challenges to be aware of
  + New language: frequent updates. This will slow down when v1.0 comes out (hopefully this year)
  + Sparse StackOverflow activity

Ongoing work:

- Forecasting under alternative monetary policy rules
- Forecast evaluation and decomposing changes in forecasts
- Estimating nonlinear models using the tempered particle filter (Herbst & Schorfheide 2017)

### Thank you!

https://github.com/FRBNY-DSGE/DSGE.jl <br>
https://github.com/FRBNY-DSGE/StateSpaceRoutines.jl